In [8]:
import numpy as np
import pandas as pd
import random

In [7]:
path = [0,7,1,2,0,12,9,0]
np.zeros(10)[9]

0.0

In [ ]:
# calculate the distance
def getdistance(path,dismatrix):
    n = len(path)
    distance = 0
    for i in range(0,len-1):
        distance += dismatrix[path[i],path[i+1]]
    return distance

# calculate invalid time
def getinvalidtime(path,timematrix,tw):
    n = len(path)
    invalidtime = 0
    ul = np.zeros(n)
    for i in range(0,n):
        invalidtime_i = 0
        if path[i] ==0:
            continue
        else:
            reachtime = ul[i-1] + timematrix[path[i-1],path[i]] + 30
            if reachtime <= tw[i,0]:
                ul[i] = tw[i,0]
            elif tw[i,0]<reachtime<=tw[i,1]: 
                ul[i] = reachtime
            else:
                ul[i] = reachtime
                invalidtime_i += reachtime - tw[i,1]
            finishtime = reachtime+30
            if finishtime > tw[i,1]:
                invalidtime_i += finishtime-tw[i,1]
        invalidtime += invalidtime_i
    
    return invalidtime

# calculate invalid vehicle
def getinvalidvehicle (path,demand,C):
    invalidcap = 0
    n = len(path)
    acumcap = np.zeros(n)
    for i in range(0,n):
        if path[i] == 0:
            continue
        else:
            acumcap[i] = acumcap[i-1] + demand[i]
    
    end = np.where(acumcap)[0]
    end = np.delete(end,0)
    for id in end:
        if acumcap[id-1]>C:
            invalidcap += C - acumcap[id-1]
    return invalidcap

In [194]:
# solution generation
path = [0,1,3,4,0,8,9,0]
def swap(path):
    n = len(path)
    point = random.sample(range(1,n-1),2)
    a = point[0]
    b = point[1]
    path[a],path[b] = path[b],path[a]
    return path

def insert(path):
    n = len(path)
    choose = random.sample(range(1,n-1),1)[0]
    location = path[choose]
    path.pop(choose)
    path.insert(random.randint(1,n-2),location)

    return path

def cross(path):
    n = len(path)
    choose = sorted(random.sample(range(1,n-1),2))
    a = choose[0]
    b = choose[1]
    route = path[a:b]
    route.reverse()
    path[a:b] = route
    
    return path

[0, 1, 3, 4, 0, 8, 9, 0]
[0, 8, 0, 4, 3, 1, 9, 0]


In [168]:
a= [1,2,7,8,5]
b = a[1:5]
b.reverse()
a[1:5]=b
a

[1, 5, 8, 7, 2]

In [191]:
c = [1,2,3]
reversed(c)